In [1]:
import io
import random
from rich.pretty import pprint
from PIL import Image
from typing import List
from inspirai_fps import Game, ActionVariable
from pyvirtualdisplay.display import Display
Display().start()

In [2]:
game = Game(
    map_dir="/mnt/d/Codes/cog-local/map-data",
    engine_dir="/mnt/d/Codes/cog-local/fps_linux_render",
)
game.init("linuxserver.x86_64")

Loaded valid locations from /mnt/d/Codes/cog-local/map-data/001.json


concatenating texture: may result in visual artifacts


Server started ...
Unity3D started ...
Unity3D connected ...


In [3]:
game.random_start_location()
game.set_map_id(random.randint(1,1))
game.set_episode_timeout(30)
game.new_episode()

pprint(game.get_game_config())

raw_frames = []

while not game.is_episode_finished():
    state = game.get_state(render_camera=True)
    raw_frames.append(state.image_bytes)
    print(f"number of frames: {len(raw_frames)}", end="\r")
    action = {
        0: [
            (ActionVariable.WALK_DIR, random.uniform(0, 360)),
            (ActionVariable.WALK_SPEED, random.uniform(0, 10)),
            (ActionVariable.TURN_LR_DELTA, 1),
        ]
    }
    game.make_action_by_list(action)

camera_frames: List[Image.Image] = [Image.open(io.BytesIO(frame)) for frame in raw_frames]

import os

img_dir = "camera_images"
os.makedirs(img_dir)

camera_frames[0].save(
    f"{img_dir}/camera.gif",
    format="GIF",
    append_images=camera_frames[1:],
    save_all=True,
    duration=100,
    loop=0,
)

Loaded valid locations from /mnt/d/Codes/cog-local/map-data/001.json
Started new episode ...
[keep] Reused map mesh from /mnt/d/Codes/cog-local/map-data/001.obj


{
│   'timeout': 30,
│   'gameMode': 'NAVIGATION_MODE',
│   'timeScale': 10,
│   'mapId': 1,
│   'randomSeed': 0,
│   'targetLocation': {'x': 1.0, 'y': 0.0, 'z': 1.0},
│   'triggerRange': 1.0,
│   'supplyHeatmapCenter': {'x': 0.0, 'y': 0.0, 'z': 0.0},
│   'supplyHeatmapRadius': 1,
│   'supplyCreatePercent': 1,
│   'supplyHouseCreatePercent': 1,
│   'supplyGridLength': 10,
│   'supplyRandomMin': 1,
│   'supplyRandomMax': 1,
│   'supplyHouseRandomMin': 10,
│   'supplyHouseRandomMax': 10,
│   'numAgents': 1,
│   'agentSetups': [
│   │   {
│   │   │   'id': 0,
│   │   │   'hp': 100,
│   │   │   'numPackAmmo': 60,
│   │   │   'gunCapacity': 15,
│   │   │   'attackPower': 20,
│   │   │   'startLocation': {'x': -72.0, 'y': 1.09, 'z': -60.0},
│   │   │   'agentName': 'agent_0'
│   │   }
│   ],
│   'respawnTime': 10,
│   'isRecord': False,
│   'replaySuffix': '',
│   'supplyLossPercentWhenDead': 50,
│   'waterSpeedDecay': 0.5,
│   'invincibleTime': 10,
│   'use_depth_map': False
}

In [4]:
for idx, img in enumerate(camera_frames):
    img.save(f"{img_dir}/img_{idx}.png")